<a href="https://colab.research.google.com/github/minxuanjun/stable-diffusion-webui-colab/blob/main/lite/stable_diffusion_inpainting_webui_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%cd /content

%env TF_CPP_MIN_LOG_LEVEL=1

!apt -y update -qq
!wget https://github.com/camenduru/gperftools/releases/download/v1.0/libtcmalloc_minimal.so.4 -O /content/libtcmalloc_minimal.so.4
%env LD_PRELOAD=/content/libtcmalloc_minimal.so.4

!apt -y install -qq aria2 libcairo2-dev pkg-config python3-dev
!pip install -q torch==2.0.1+cu118 torchvision==0.15.2+cu118 torchaudio==2.0.2+cu118 torchtext==0.15.2 torchdata==0.6.1 --extra-index-url https://download.pytorch.org/whl/cu118 -U
!pip install -q xformers==0.0.20 triton==2.0.0 gradio_client==0.2.7 -U

!git clone -b v2.4 https://github.com/camenduru/stable-diffusion-webui
!git clone https://huggingface.co/embed/negative /content/stable-diffusion-webui/embeddings/negative
!git clone https://huggingface.co/embed/lora /content/stable-diffusion-webui/models/Lora/positive
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/embed/upscale/resolve/main/4x-UltraSharp.pth -d /content/stable-diffusion-webui/models/ESRGAN -o 4x-UltraSharp.pth
!wget https://raw.githubusercontent.com/camenduru/stable-diffusion-webui-scripts/main/run_n_times.py -O /content/stable-diffusion-webui/scripts/run_n_times.py
!git clone -b v2.4 https://github.com/camenduru/deforum-for-automatic1111-webui /content/stable-diffusion-webui/extensions/deforum-for-automatic1111-webui
!git clone -b v2.4 https://github.com/camenduru/stable-diffusion-webui-images-browser /content/stable-diffusion-webui/extensions/stable-diffusion-webui-images-browser
!git clone -b v2.4 https://github.com/camenduru/stable-diffusion-webui-huggingface /content/stable-diffusion-webui/extensions/stable-diffusion-webui-huggingface
!git clone -b v2.4 https://github.com/camenduru/sd-civitai-browser /content/stable-diffusion-webui/extensions/sd-civitai-browser
!git clone -b v2.4 https://github.com/camenduru/sd-webui-additional-networks /content/stable-diffusion-webui/extensions/sd-webui-additional-networks
!git clone -b v2.4 https://github.com/camenduru/sd-webui-tunnels /content/stable-diffusion-webui/extensions/sd-webui-tunnels
!git clone -b v2.4 https://github.com/camenduru/batchlinks-webui /content/stable-diffusion-webui/extensions/batchlinks-webui
!git clone -b v2.4 https://github.com/camenduru/stable-diffusion-webui-catppuccin /content/stable-diffusion-webui/extensions/stable-diffusion-webui-catppuccin
!git clone -b v2.4 https://github.com/camenduru/stable-diffusion-webui-rembg /content/stable-diffusion-webui/extensions/stable-diffusion-webui-rembg
!git clone -b v2.4 https://github.com/camenduru/stable-diffusion-webui-two-shot /content/stable-diffusion-webui/extensions/stable-diffusion-webui-two-shot
!git clone -b v2.4 https://github.com/camenduru/sd-webui-aspect-ratio-helper /content/stable-diffusion-webui/extensions/sd-webui-aspect-ratio-helper
!git clone -b v2.4 https://github.com/camenduru/asymmetric-tiling-sd-webui /content/stable-diffusion-webui/extensions/asymmetric-tiling-sd-webui
%cd /content/stable-diffusion-webui
!git reset --hard
!git -C /content/stable-diffusion-webui/repositories/stable-diffusion-stability-ai reset --hard

!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/i15/resolve/main/sd-v1-5-inpainting.ckpt -d /content/stable-diffusion-webui/models/Stable-diffusion -o sd-v1-5-inpainting.ckpt

!sed -i -e '''/from modules import launch_utils/a\import os''' /content/stable-diffusion-webui/launch.py
!sed -i -e '''/        prepare_environment()/a\        os.system\(f\"""sed -i -e ''\"s/dict()))/dict())).cuda()/g\"'' /content/stable-diffusion-webui/repositories/stable-diffusion-stability-ai/ldm/util.py""")''' /content/stable-diffusion-webui/launch.py
!sed -i -e 's/\["sd_model_checkpoint"\]/\["sd_model_checkpoint","sd_vae","CLIP_stop_at_last_layers"\]/g' /content/stable-diffusion-webui/modules/shared.py

!python launch.py --listen --xformers --enable-insecure-extension-access --theme dark --gradio-queue --multiple

In [ ]:
import torch
import diffusers

import torch
from diffusers import StableDiffusionPipeline
base_model = "runwayml/stable-diffusion-v1-5"

pipe = StableDiffusionPipeline.from_pretrained(base_model, torch_dtype=torch.float16)

the pre-trained model includes all the components requireed to setup a complete diffusion pipeline. They are stored in the folloing folders:
- `text_encoder`: Stable Diffusion used CLIP, but other diffusion models may use other encoders such as `BERT`.
- `tokenizer`: It must match the one used by the `text_encoder` model.
- `scheduler`: The scheduling algorithm used to progressively add noise to the image during tranning.
- `unet`: The model used to generate the latent representation of the input.
- `vae`: Autoencoder module that we'll use to decode latent representations into real images.

We can load the components by referring to the folder they were saved, using `subfolder` argument to `from_pretrained`.  

In [ ]:
print(pipe.components)
print(type(pipe.components))


In [ ]:
pipe = pipe.to("cuda")

## CLip tokenizer [Text Encoder tutorail](https://zhuanlan.zhihu.com/p/680103276)

CLIPTokenizer 会将文本拆分成各个单词，然后使用查表法将每个子单词转换为数字。
其中每个prompt 编码的最大长度是77。每个编码出的序列都有一些固定的token. *start_token*, *end_token*.
```python
'tokenizer': CLIPTokenizer(name_or_path='/root/.cache/huggingface/hub/models--runwayml--stable-diffusion-v1-5/snapshots/f03de327dd89b501a01da37fc5240cf4fdba85a1/tokenizer', vocab_size=49408, model_max_length=77, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<|startoftext|>', 'eos_token': '<|endoftext|>', 'unk_token': '<|endoftext|>', 'pad_token': '<|endoftext|>'
    }, clean_up_tokenization_spaces=False),  added_tokens_decoder={
        49406: AddedToken("<|startoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
        49407: AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
    }
```

In [ ]:
tokenizer = pipe.components["tokenizer"]
print(type(tokenizer))
prompt = ["a dog wearning hat"]
prompt = ['tokenizer = pipe.components["tokenizer"]']
text_token = tokenizer(prompt, padding="max_length", max_length = tokenizer.model_max_length, truncation=True, return_tensors = "pt")
text_token = text_token.to("cpu")
print(text_token.input_ids.shape)
print(text_token)

for token in list(text_token.input_ids[0,:7]):
  print(f"{token}:{tokenizer.convert_ids_to_tokens(int(token.item()))}")

## Text Encoder
Text Encoder 就是将将tokenizer 生成的input_ids 转化为一个文本嵌入的特征，文本嵌入的特征维度是768。Text Encoder 就是， 输出特征维度是[77,768]

```python
'text_encoder': CLIPTextModel(
  (text_model): CLIPTextTransformer(
    (embeddings): CLIPTextEmbeddings(
      (token_embedding): Embedding(49408,
    768)
      (position_embedding): Embedding(77,
    768)
    )
    (encoder): CLIPEncoder(
      (layers): ModuleList(
        (0-11): 12 x CLIPEncoderLayer(
          (self_attn): CLIPAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=True)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (layer_norm1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (mlp): CLIPMLP(
            (activation_fn): QuickGELUActivation()
            (fc1): Linear(in_features=768, out_features=3072, bias=True)
            (fc2): Linear(in_features=3072, out_features=768, bias=True)
          )
          (layer_norm2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        )
      )
    )
    (final_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
)
```

In [ ]:
text_encoder = pipe.components["text_encoder"]
# get model structure
print(text_encoder)

with torch.no_grad():
  text_embed = text_encoder(text_token.input_ids.to("cuda"))
# print(text_embed)
print(f"hidden {text_embed.last_hidden_state.shape}  {text_embed.pooler_output.shape}")


我们可以通过控制  `num_inference_steps` 来控制去噪次数，去噪次数越多，结果越好。 默认值是50.

In [ ]:
prompt = "a photograph of an astronaut riding a horse"
%debug image = pipe(prompt, num_inference_steps=15).images[0]  # image here is in [PIL format](https://pillow.readthedocs.io/en/stable/)

# Now to display an image you can either save it such as:
image.save(f"astronaut_rides_horse.png")

# or if you're in a google colab you can directly display it with
image

The other parameter in the pipeline call is guidance_scale. It is a way to increase the adherence to the conditional signal which in this case is text as well as overall sample quality. In simple terms classifier free guidance forces the generation to better match with the prompt. Numbers like 7 or 8.5 give good results, if you use a very large number the images might look good, but will be less diverse.

To generate multiple images for the same prompt,

1.   List item
2.   List item

we simply use a list with the same prompt repeated several times. We'll send the list to the pipeline instead of the string we used before.



Let's first write a helper function to display a grid of images. Just run the following cell to create the `image_grid` function, or disclose the code if you are interested in how it's done.

In [ ]:
from PIL import Image

def image_grid(imgs, rows, cols):
    assert len(imgs) == rows*cols

    w, h = imgs[0].size
    grid = Image.new('RGB', size=(cols*w, rows*h))
    grid_w, grid_h = grid.size

    for i, img in enumerate(imgs):
        grid.paste(img, box=(i%cols*w, i//cols*h))
    return grid

**Now**, we can generate a grid image once having run the pipeline with a list of 3 prompts.

In [ ]:
num_images = 12
prompt = ["a photograph of an astronaut riding a horse"] * num_images

images = pipe(prompt).images

grid = image_grid(images, rows=4, cols=3)
grid

What is Stable Diffusion
Now, let's go into the theoretiacal part of Stable Diffusion.

Stable Diffusion is based on a particular type of diffusion model call **Latent Diffusion**, proposed in [High-Resolution Image Synthesis with Latent Diffusion Models](https://arxiv.org/abs/2112.10752).


General diffusion models are machine learning systems that are trained to denoise random gaussian noise step by step, to get to a sample of interest, such as an image, For a more detaied overview of how they work, check [this colab](https://colab.research.google.com/github/huggingface/notebooks/blob/main/diffusers/diffusers_intro.ipynb).

Diffusion models have shown to achieve state-of-the-art results for generating image data, But one downside of diffusion models is that the reverse denoising process is slow. In addition, these models consume a lot of memory because they operate in pixel space, which becomes unreasonably expensive when generating high-resolution images. Therefore, it is challenging to tain these models and also use them for inference.

Latent diffusion can reduce memory and compute complexity by applying the diffusion process over a lower dimensional latent space, instead of using the actual pixel space. This is the key difference between standard diffusion and latent diffusion models: in latent diffusion the model is trained to generate latent (compressed) representations of the images.

There are three main components in latent diffusion.

1. An autoencoder (VAE).
2. A [U-Net](https://colab.research.google.com/github/huggingface/notebooks/blob/main/diffusers/diffusers_intro.ipynb#scrollTo=wW8o1Wp0zRkq).
3. A text-encoder, *e.g.* [CLIP's Text Encoder](https://huggingface.co/docs/transformers/model_doc/clip#transformers.CLIPTextModel).


**1. The autoencoder(VAE)**

The VAE model has two parts, an encoder and a decoder. The encoder is used to convert the image into a low dimensional latent representation, which will serve as the input to the *U-Net* model.
Tne decoder, conversely, transforms the laten representation back into an image.

During latent diffusion _training_, the encoder is used to get the latent representation (_latens_) of the images for the forward diffusion process, which applies more and more noise at each step.
During _inference_, the denoised latents generated by the reverse diffusion process are converted back into images using the VAE decoder. As will see during inference we only need the VAE dencoder.


**2. The U-Net**
The U-Net has encoder part and decoder part both comprised of ResNet blocks. The encoder compresses an image representation into a lower resolution image representation and the decoder decodes the lower resolution image representation back to the original higher image representation that is supposedly less noisy. More specifically, the U-Net output predicts the *noise residual* which can be used to compute the denoised image representation.

To prevent the U-Net from losing important information while downsampling, short-cut connetctions are usuallig added between the downsampling ResNets of the encoder to the upsampling ResNets of the decoder.
Additionally, the stable diffusion U-Net is able to condition its output on text-enbeddding via cross-attention layers. The cross-attention layers are added to both the encoder and decoder part of the U-Net usually between ResNet blocks.

**3. The Text-endoer**
The text-encoder is reponsible for transformiing the input prompt, *e.g." "An astronout riding a horse" into an embedding space that can be understand by the U-Net. It is usually  a simple *transformer-based* encoder that maps a sequnce of input tokens to a sequnce of latent text-embeddings.

Inspired by [Image](https://imagen.research.google/), Stable Diffusion does **not** train the text-encoder during trainning and simply use an CLIP's already trained text encoder,  [CLIPTextModel](https://huggingface.co/docs/transformers/model_doc/clip#transformers.CLIPTextModel).

**Why is latent diffusion fast and efficient?**

Since the U-Net latent diffusion models operators on a low dimensional space, it greatly reducesthe memory and compute requirements compared to pixel-spce diffusion models.

# How to write your own inference pipeline with 'diffusers'

Finally, we show how you can create custom diffusion pipelines with `diffusers`.
In this section, we will demonstrate how to use Stable Diffusion with a different scheduler, namely [Katherine Crowson's](https://github.com/crowsonkb) K-LMS scheduler that was added in [this PR](https://github.com/huggingface/diffusers/pull/185#pullrequestreview-1074247365).

Let's go through the `StableDiffusionPipeline` step by step to see how we cloud have written its ourselves.
We will start by loading the individual models involved.  

In [ ]:
import torch
from transformers import CLIPTextModel, CLIPTokenizer
from diffusers import AutoencoderKL, UNet2DConditionModel, PNDMScheduler
torch_device = "cuda" if torch.cuda.is_available() else "cpu"
base_model = "runwayml/stable-diffusion-v1-5"
torch_dtype = torch.float32
# 1. Load the autoencoder model with will be used to decode the latents into image space.
vae = AutoencoderKL.from_pretrained(base_model, subfolder="vae", torch_dtype= torch_dtype)
# 2. Load the tokenizer and text encoder to tokenizer and encode the text.
tokenizer = CLIPTokenizer.from_pretrained("openai/clip-vit-large-patch14", torch_dtype = torch_dtype)
text_encoder = CLIPTextModel.from_pretrained("openai/clip-vit-large-patch14", torch_dtype= torch_dtype)

# 3. The UNet model for generating the latents
unet =UNet2DConditionModel.from_pretrained(base_model, subfolder='unet', torch_dtype=torch_dtype)

Now instead of loading the pre-defined scheduler, we'll use the K-LMS scheduler instead.

In [ ]:
from diffusers import LMSDiscreteScheduler

scheduler = LMSDiscreteScheduler.from_pretrained(base_model, subfolder="scheduler", torch_dtype=torch_dtype)

Next we move the models to the GPU.

In [ ]:
vae = vae.to(torch_device)
text_encoder = text_encoder.to(torch_device)
unet= unet.to(torch_device)

We now define the parameters we'll use to generate images.
Note that `guidance_scale = 1` corresponds to doing no classifier-free guaidance.
Here we set it to7.5 as also done previously.
In contrast to the previous examples, we set `num_inference_steps` to 100 to get an even more defined image.

In [ ]:
prompt = ["a photograph of an astronaut riding a horse"]

height = 512
width= 512

num_inference_steps = 100
guidance_scale = 7.5
generator = torch.manual_seed(32)

batch_size = 1


First, we get the text_embedding for the prompt. These embeddingswill be used to condition the UNet model.

In [ ]:
text_input = tokenizer(prompt, padding="max_length", max_length=tokenizer.model_max_length, truncation=True, return_tensors="pt")

with torch.no_grad():
  text_embeddings = text_encoder(text_input.input_ids.to(torch_device))[0]

We'll also get the unconditional text embedding for classifier-free guidance, which are just the embedding for the padding token(empty text).
They need to have the same shape as the conditional `text_embeddings` (`batch_size` and `seq_length`)

In [ ]:
max_length =text_input.input_ids.shape[-1]
print(max_length)
uncond_input = tokenizer([""]* batch_size, padding="max_length", max_length=max_length, return_tensors="pt")

with torch.no_grad():
  uncond_embeddings = text_encoder(uncond_input.input_ids.to(torch_device))[0]

For classifier-free guidance, we need to do two forward passes. Once with the conditioned input(`text_embeddings`), and another with unconditional embedding (`uncond_embeddings`) In practice, we can concatenate both into a single batch to avoid doing two forward passes.

In [ ]:
text_embeddings= torch.cat([uncond_embeddings, text_embeddings])
print(text_embeddings.shape)

Generate the Initial random noise

In [ ]:
latents = torch.randn((batch_size, unet.config.in_channels, height//8, width//8))
print(latents.shape)

Cool $64 \times 64$ is expected. The model will transform this latent representation (pure noise) into a `512 × 512` image later on.

Next, we initialize the scheduler with our chosen `num_inference_steps`.
This will compute the `sigmas` and exact time step values to be used during the denoising process.


In [ ]:
scheduler.set_timesteps(num_inference_steps)

The K-LMS scheduler needs to multiply the `latents` by its `sigma` values. Let's do this here

In [ ]:
latents = latents * scheduler.init_noise_sigma

We are ready to write the denoising loop.

In [ ]:
from tqdm.auto import tqdm
from torch import autocast

for t in tqdm(scheduler.timesteps):
  # expand the latents if we are doing classifier-free guidance to avoid doing two forward passes.
  latent_model_input = torch.cat([latents] * 2)

  latent_model_input = scheduler.scale_model_input(latent_model_input, t)

  # predict the noise residual
  with torch.no_grad():
    noise_pred = unet(latent_model_input, t, encoder_hidden_states=text_embeddings).sample

  # perform guidance
  noise_pred_uncond, noise_pred_text = noise_pred.chunk(2)
  noise_pred = noise_pred_uncond + guidance_scale * (noise_pred_text - noise_pred_uncond)

  # compute the previous noisy sample x_t -> x_t-1
  latents = scheduler.step(noise_pred, t, latents).prev_sample

We now use the `vae` to decode the generated `latents` back into the image.

In [ ]:
# scale and decode the image latents with vae
latents = 1 / 0.18215 * latents

with torch.no_grad():
  image = vae.decode(latents).sample

And finally, let's convert the image to PIL so we can display or save it.

In [ ]:
image = (image / 2 + 0.5).clamp(0, 1)
image = image.detach().cpu().permute(0, 2, 3, 1).numpy()
images = (image * 255).round().astype("uint8")
pil_images = [Image.fromarray(image) for image in images]
pil_images[0]